## SQL Queries 

### Start with simple hypothetical examples, then move to triggers, procedures, functions, sequences.

In [ ]:
# Check tables within the schema, after logging on
select TABLESPACE_NAME, TABLE_NAME 
from USER_TABLES;

# Check all tables accessible to current user
select TABLESPACE_NAME, TABLE_NAME 
from ALL_TABLES;

# Check an individual table
desc 'TABLENAME';

In [ ]:
### Create table for Region & Country combined as there is no ID number for Region, therefore no way
### of linking separate Country & Region tables

In [ ]:
create TABLE Country AS
select DISTINCT Country, Region, ISO2, ISO3
from(
select * 
from COUNTRYREGION) C;

In [ ]:
select count(*), Country
from COUNTRY
group by COUNTRY;

### 206 Countries, no duplicates
### Primary Key: ISO2 in Country table, Foreign Key: ISO2 in Main table

In [ ]:
alter TABLE Country 
    modify (CONSTRAINT main_pk PRIMARY KEY (ISO2));  # Turn existing field into PRIMARY KEY
    
alter TABLE MAIN 
  modify ISO2 CONSTRAINT main_fk REFERENCES Country(ISO2);  # Turn existing field into FOREIGN KEY

In [ ]:
create SEQUENCE main_seq START WITH 1; 
### Creating sequence to create ID for Main table as auto-incrementing PRIMARY KEY

In [ ]:
# Triggers are stored programs, which are automatically executed or fired when some events occur. 
# One very good reason for this could be auditing     "AFTER"

In [ ]:
create OR replace trigger PHONE_BOOK_HISTORY
    after update ON PHONE_BOOK                              ### AFTER ###

begin
    insert into PHONE_BOOK_EDIT_HISTORY

    (AUDIT_HISTORY_ID, USERNAME, MODIFICATION, EDIT_DATE)    # columns in "PHONE_BOOK_EDIT_HISTORY"
     values
    (AUDIT_HISTORY_ID_SEQUENCE.nextVAL, USER, 'UPDATE', SYSDATE);   ### values to be inserted

end;

In [ ]:
# Triggers can be also created to enforce constraints. "BEFORE"

In [ ]:
create trigger PHONEBOOK
    before insert ON someuser.PHONE_BOOK                 ### BEFORE ####

begin
    raise_application_error (                           ### if application error raised, record will
         num => -20050,                                 ### not be inserted
         msg => 'Error message goes here.');
end;

In [ ]:
create TRIGGER IDX                     ### Creating TRIGGER for IDX primary key to auto-increment.
    before INSERT on MAIN              ### BEFORE ###
    for EACH ROW

begin
  select main_seq.NEXTVAL
  into   ID
  from   dual;
end;
/

## Now look at existing schema within SQL Developer 

In [ ]:
### This is just to have a good understanding of what is in the database and what columns the tables
### are made of.

select TABLESPACE_NAME, TABLE_NAME 
from USER_TABLES;

USERS	REGIONS
USERS	LOCATIONS
USERS	DEPARTMENTS
USERS	JOBS
USERS	EMPLOYEES
USERS	JOB_HISTORY


In [ ]:
desc REGIONS;  ### look at the "Regions" table

Name        Null     Type         
----------- -------- ------------ 
REGION_ID   NOT NULL NUMBER       
REGION_NAME          VARCHAR2(25) 

desc LOCATIONS;   ### look at the "Locations" table

Name           Null     Type         
-------------- -------- ------------ 
LOCATION_ID    NOT NULL NUMBER(4)    
STREET_ADDRESS          VARCHAR2(40) 
POSTAL_CODE             VARCHAR2(12) 
CITY           NOT NULL VARCHAR2(30) 
STATE_PROVINCE          VARCHAR2(25) 
COUNTRY_ID              CHAR(2) 

desc DEPARTMENTS;   ### look at the "Departments" table

Name            Null     Type         
--------------- -------- ------------ 
DEPARTMENT_ID   NOT NULL NUMBER(4)    
DEPARTMENT_NAME NOT NULL VARCHAR2(30) 
MANAGER_ID               NUMBER(6)    
LOCATION_ID              NUMBER(4) 

desc JOBS;         ### look at the "Jobs" table

Name       Null     Type         
---------- -------- ------------ 
JOB_ID     NOT NULL VARCHAR2(10) 
JOB_TITLE  NOT NULL VARCHAR2(35) 
MIN_SALARY          NUMBER(6)    
MAX_SALARY          NUMBER(6) 

desc EMPLOYEES     ### look at the "Employees" table

Name           Null     Type         
-------------- -------- ------------ 
EMPLOYEE_ID    NOT NULL NUMBER(6)    
FIRST_NAME              VARCHAR2(20) 
LAST_NAME      NOT NULL VARCHAR2(25) 
EMAIL          NOT NULL VARCHAR2(25) 
PHONE_NUMBER            VARCHAR2(20) 
HIRE_DATE      NOT NULL DATE         
JOB_ID         NOT NULL VARCHAR2(10) 
SALARY                  NUMBER(8,2)  
COMMISSION_PCT          NUMBER(2,2)  
MANAGER_ID              NUMBER(6)    
DEPARTMENT_ID           NUMBER(4)

desc JOB_HISTORY   ### look at the "Job History" table

Name          Null     Type         
------------- -------- ------------ 
EMPLOYEE_ID   NOT NULL NUMBER(6)    
START_DATE    NOT NULL DATE         
END_DATE      NOT NULL DATE         
JOB_ID        NOT NULL VARCHAR2(10) 
DEPARTMENT_ID          NUMBER(4)  


## Script to find Primary KEY

In [ ]:
set verify off
accept TABLE_NAME char prompt 'Table name>'

select COLS.COLUMN_NAME                
from ALL_CONSTRAINTS as CONS              ### Check PK/FK keys to understand relationships better.
NATURAL JOIN
ALL_CONS_COLUMNS as COLS
WHERE cons.constraint_type = 'P' AND table_name = UPPER('&TABLE_NAME');

EMPLOYEES table:
--- "EMPLOYEE_ID" 
JOBS table:
--- "JOB_ID"
REGIONS table:
--- "REGION_ID"
LOCATIONS table:
--- "LOCATION_ID"
DEPARTMENTS table:
--- "DEPARTMENT_ID"
JOB_HISTORY table:                        ### PK is complex key.
--- "EMPLOYEE_ID"  
--- "START_DATE"

### Demonstrate INNER join and subquery example
### "Show all employees that are Managers"

In [ ]:
select EMP.FIRST_NAME||' '||EMP.LAST_NAME as Fullname, JOBS.JOB_TITLE
from                                                                    ### INLINE VIEW ###
(select * 
from EMPLOYEES
where COMMISSION_PCT is null) EMP                   ### List of employees not paid commission 
inner join                                          ### Join is possile with Subqueries
JOBS
on EMP.JOB_ID = JOBS.JOB_ID
ORDER BY JOBS.JOB_TITLE;

In [ ]:
### Subquery in the "FROM" clause of the "SELECT" statement, the "INLINE VIEW"
### Subquery in the "WHERE" clause of the "SELECT" statement, the "NESTED SUBQUERY"
### A subquery can also be nested inside INSERT, UPDATE, and DELETE statements.

In [ ]:
select FIRST_NAME||' '||LAST_NAME as FullName, SALARY, DEPARTMENT_ID
from EMPLOYEES
where SALARY = (select MIN(SALARY) from EMPLOYEES);     ### NESTED SUBQUERY
                                                        ### only show employee with lowest salary
select DEPARTMENT_ID, MIN(SALARY)
from EMPLOYEES                                          ### employees with salary lower than average
group by DEPARTMENT_ID
having MIN(SALARY) < (select AVG(SALARY) from EMPLOYEES);      

In [ ]:
### Correlated Subquery is where the value of the inner subquery depends on the value of the outer 
### subquery
### This means that in a correlated subquery (usually not efficient for large datasets)

In [ ]:
select FIRST_NAME||' '||LAST_NAME as FullName, JOB_ID, SALARY
from EMPLOYEES
where (SALARY, DEPARTMENT_ID)
in                                              ## Using "IN" because returning several rows from
(select SALARY, DEPARTMENT_ID                   ## subquery
 from EMPLOYEES
 where SALARY between 1000 and 20000
 and
 DEPARTMENT_ID between 10 and 20)               ## Selecting employees with particular salary range
order by FullName;                              ## and from particular departments

## Now look at SQL Oracle Objects in general

In [ ]:
### List of objects:
### table, view, synonym, index, package, package body, sequence, type body, function, procedure, 
### trigger, type, library, materialised view, materialised view log, database link

### Synonym: generally use them when granting access to an object from another schema and 
### not worried about owner of object.

In [ ]:
create PUBLIC synonym EMPLOYEES
for HR_Oracle.EMPLOYEES;

# Create, replace, drop

### Index: allows faster retrieval of records. Generally good practice to use an index on larger tables, 
### can make a big difference to performance.

In [ ]:
create index EMPLOYEES_IX
    on EMPLOYEES(SALARY)  # NOT an ideal type to index, "EMPLOYEE_ID", "JOB_ID", "DEPARTMENT_ID", 
                          # "MANAGER_ID" would be better but they are already CONSTRAINTS.
    COMPUTE STATISTICS;   # Collect statistics.

# Can't index columns that are PRIMARY or FOREIGN KEYS already.

### Sequence: to create autonumbers (often ideal for PRIMARY keys)

In [ ]:
create sequence EMPLOYEE_SEQ
  MINVALUE 1
  MAXVALUE 1000000
  START WITH 1
  INCREMENT BY 1
  CACHE 20;               ### The number looking to cache at a particular time.
    
EMPOLYEE_SEQ.nextval;     ### This is used to retrieve a particular value using a sequence. 
                          ### An example of its usefulness is when looking to insert records.

### Procedure: uses comparison operator 'LIKE' to search for pattern in text

In [ ]:
declare   

procedure
    compare (value VARCHAR2, pattern VARCHAR2) is      ### calls procedure
    begin 
    if value LIKE pattern
          then
          dbms_output.put_line('FOUND IT!');
    else
          dbms_output.put_line('keep searching');
    
    end if;
  
    end;   
  
    begin
  
        compare('Taylor Smith', 'T% S%');  ## compares 'Taylor Smith' to pattern 'T''S' (valid)
        compare('Margaret Smith', 'T% S%');  ## compares 'Margaret Smith' to pattern 'T''S' 
                                             ## (invalid)
  end;
/
********************
PL/SQL procedure successfully completed.

FOUND IT!
keep searching
********************

### Procedure using existing Pipeline Opportunity (for example) - updating the OPPORTUNITY table

In [ ]:
create procedure UPDATE_OPPORTUNITY
    (OPP_ID number, STATUS, varchar2) as
    
    STATUS_FINAL real;             ### if "STATUS_FINAL" is not null
    STATUS_MISSING exception;      ### raises error if "STATUS_FINAL" is null
    
    begin
        select OPP_STATUS into STATUS_FINAL from OPPORTUNITY
        where OPPORTUNITY_ID = OPP_ID;
        if STATUS_FINAL is null
            then
            raise STATUS_MISSING;            ### raises error as noted above
        else
            update OPPORTUNITY
            set OPPORTUNITY_STATUS = 'Closed Lost'   
            where OPPORTUNITY_ID = OPP_ID    ### Specifies the conditions on when to 
            and                              ### update "OPPORTUNITY" table
            STATUS = 'Lost';
            
        end if;
        exception                             ### Raising exception for procedure
            when STATUS_MISSING
            then
                dbms_output.put_line('This opportunity will not be updated');
                OPPORTUNITY_STATUS := 'NA';
                    
            when OTHERS then
                NULL;                       ### Exception to do nothing.
                
end UPDATE_OPPORTUNITY;
/

In [ ]:
call UPDATE_OPPORTUNITY (2555203669, 'Lost');       ## Call procedure to be performed.

In [ ]:
### You can use a procedure in "DECLARE" and execute it as part of a subprogram.

### Using Condition, a simple IF statement for updating a table

In [ ]:
### Updating the EMPLOYEES table with salary for employee number 100

declare
  E_ID EMPLOYEES.EMPLOYEE_ID%TYPE := 100;  ## will update this record if condition is True.
  E_SAL EMPLOYEES.SALARY%TYPE;             ## %TYPE is to make sure column has same type as 
                                           ## original table
begin
  select SALARY
  into E_SAL
  from EMPLOYEES
  where EMPLOYEE_ID = E_ID;
  
    if (E_SAL < 30000) then                ## if salary is less than 30,000 then EMPLOYEES table 
      update EMPLOYEES                     ## is updated
                                           
      set SALARY = SALARY + 10000
        where
        EMPLOYEE_ID = E_ID;
        
        dbms_output.put_line('Salary Updated');
        
     end if;
    
end;
/

### Using "CASE" for a simple salary band comparison

In [ ]:
declare

  E_ID EMPLOYEES.EMPLOYEE_ID%TYPE := 100;
  E_SAL EMPLOYEES.SALARY%TYPE;
  
  begin
  
    select SALARY
    into E_SAL
    from EMPLOYEES
    where EMPLOYEE_ID = E_ID;
    
    case
    
      when E_SAL < 10000 then dbms_output.put_line('too low ' || E_SAL);
      when E_SAL >= 10000 and E_SAL < 30000 then dbms_output.put_line('average '|| E_SAL);
      when E_SAL >= 30000 then dbms_output.put_line('good '|| E_SAL);
      
    else
      dbms_output.put_line('off scale '|| E_SAL);
      
    end case;
    
  end;
  
/

### Loops

In [ ]:
### A very simple loop with "exit" instead of using "IF/ELSIF/ELSE"
declare
    x number := 10;
  
    begin
        loop
        dbms_output.put_line(x);
      
            x := x + 5;
      
        exit when x > 50;
    end loop;
  end;
/

######################################################################

### Same as above, except using "WHILE" loop.

declare
    x number := 10;
    begin
        while x <= 50 loop
    
          dbms_output.put_line(x);
        
          x := x + 5;
    end loop;
  end;
/

In [ ]:
### For loop - simple, showing reverse order and keyword positionining

declare
    a number (2);
    begin
    for a in 10 .. 20 loop
  
        dbms_output.put_line('looping: '||a);
    
    end loop;
end;
/
######################################################

declare
    a number (2);
    begin
    for a in reverse 10 .. 20 loop      ### counting within loop is reversed
  
        dbms_output.put_line('looping: '||a);
    
  end loop;
end;
/

In [ ]:
### Cursor: retrieves particular attributes selected under 'declare'. 

In [ ]:
### Function: this is an example of locating the Empolyee Number for a particular Employee

In [ ]:
### *** UNFINISHED ***

create FUNCTION FindEmployee
   (name_in IN VARCHAR2)
   return number
IS
   number;

   cursor E1 is
   select EMPLOYEE_ID
     from EMPLOYEES
     where LAST_NAME = name_in;

begin

   open E1;
   fetch E1 into EMPnumber;

   if E1%notfound then
      EMPnumber := 9999999;
   end if;

   close E1;

return EMPnumber;

exception
when OTHERS then
   raise_application_error(-20001,'An error was encountered - '||SQLCODE||' -ERROR- '||SQLERRM);
end;
/

In [ ]:
### Make sure output is printed 

SET SERVEROUTPUT ON;

In [ ]:
### Drop Function (or any other object, of course)

DROP FUNCTION FindEmployee;